In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [2]:
import datetime as dt
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
import scipy.sparse
train = pd.read_csv("../input/train.csv")
train['date'] = pd.to_datetime(train['date'], errors='coerce')
train['year'] = train['date'].dt.year
train['year'].replace({2013:1,2014:2,2015:3,2016:4,2017:5},inplace=True)
train['month'] = train['date'].dt.month
train['dayofweek'] = train['date'].dt.dayofweek
train['weekofyear'] = train['date'].dt.weekofyear
test = pd.read_csv("../input/test.csv")
test['date'] = pd.to_datetime(test['date'], errors='coerce')
test['year'] = test['date'].dt.year
test['year'].replace({2018:6},inplace=True)
test['month'] =test['date'].dt.month
test['dayofweek'] = test['date'].dt.dayofweek
test['weekofyear'] = test['date'].dt.weekofyear
pred_list = []
import lightgbm as lgb
clf = lgb.LGBMRegressor()
for i in range(1,51):
    train_X = train[(train['item']==i)&(train['year']<6)].drop(columns=['sales','date'])
    train_y = train[(train['item']==i)&(train['year']<6)]['sales']
    test_X = test[(test['item']==i)&(test['year']==6)].drop(columns=['date','id'])
    clf.fit(train_X, train_y)
    pred_y = clf.predict(test_X)
    pred_y = pred_y.tolist()
    pred_list.extend(pred_y)

In [3]:
s = test[['id']]
s['sales'] = pred_list
s.to_csv('upload.csv',index=0)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
